<a href="https://colab.research.google.com/github/deepesh321/Sentiment_classification/blob/master/sentiment_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
BASE_DIR='/content/drive/My Drive/'

In [0]:
import pandas as pd
import numpy as np
df =pd.read_csv(BASE_DIR+'text_train.csv',names=[
                 'image_name', 'funny_image', 'sarcastic_image', 'offensive_image', 'motivational_image', 'text'])

In [4]:
df.head(3)

,image_name,funny_image,sarcastic_image,offensive_image,motivational_image,text
0,10_year_2r94rv.jpg,funny,sarcastic,not_offensive,not_motivational,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...
1,10_year_10-year-challenge_1547788782.jpeg,not_funny,sarcastic,not_offensive,motivational,The best of #10 YearChallenge! Completed in le...
2,10_year_10yearchallenge-5c75f8b946e0fb0001edc7...,funny,not_sarcastic,not_offensive,not_motivational,Sam Thorne @Strippin ( Follow Follow Saw every...


In [0]:
def clean_text(text):

    # Remove puncuation
    text = text.translate(string.punctuation)

    # Convert words to lower case and split them
    text = text.lower().split()

    # Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]

    text = " ".join(text)

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)

    return text


In [6]:
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
from nltk.stem import SnowballStemmer
df['text'] = df['text'].map(lambda x: clean_text(x))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
df.head(3)

,image_name,funny_image,sarcastic_image,offensive_image,motivational_image,text
0,10_year_2r94rv.jpg,funny,sarcastic,not_offensive,not_motivational,look friend lightyear sohalikut trend play yea...
1,10_year_10-year-challenge_1547788782.jpeg,not_funny,sarcastic,not_offensive,motivational,best 10 yearchalleng ! complet less year kudus...
2,10_year_10yearchallenge-5c75f8b946e0fb0001edc7...,funny,not_sarcastic,not_offensive,not_motivational,sam thorn strippin follow follow saw everyon p...


In [0]:
labels = []
for index, row in df.iterrows():
    if row['funny_image'] == 'funny':
        row['funny_image'] = 1
    else:
        row['funny_image'] = 0
    if row['sarcastic_image'] == 'sarcastic':
        row['sarcastic_image'] = 1
    else:
        row['sarcastic_image'] = 0
    if row['offensive_image'] == 'offensive':
        row['offensive_image'] = 1
    else:
        row['offensive_image'] = 0
    if row['motivational_image'] == 'motivational':
        row['motivational_image'] = 1
    else:
        row['motivational_image'] = 0
    labels.append([row['funny_image'], row['sarcastic_image'],
                   row['offensive_image'], row['motivational_image']])


In [9]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [0]:
#In this approach we learn word embedding as part of fitting a neural network model.
tokenizer_obj=Tokenizer()
#print(tokenizer_obj)
tokenizer_obj.fit_on_texts(df['text'])

In [0]:
max_length=max([len(s.split()) for s in df['text']])

In [0]:
vocab_size=len(tokenizer_obj.word_index)+1
#print(tokenizer_obj.word_index)

In [0]:
data_tokens=tokenizer_obj.texts_to_sequences(df['text'])
data=pad_sequences(data_tokens,maxlen=max_length,padding='post')

In [0]:
from keras.models import Sequential
from keras.layers import Dense,LSTM,GRU
from keras.layers.embeddings import Embedding
from keras.optimizers import SGD
import keras

In [15]:
model=Sequential()
model.add(Embedding(vocab_size,100,input_length=max_length))
model.add(GRU(units=64,dropout=0.2))
model.add(Dense(4,activation='sigmoid'))
lr_schedule = keras.callbacks.LearningRateScheduler(lambda epoch: 1e-8 * 10**(epoch / 10))
opt = SGD(lr=1e-8, momentum=0.9)
model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [16]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1316, 100)         1111000   
_________________________________________________________________
gru_1 (GRU)                  (None, 64)                31680     
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 260       
Total params: 1,142,940
Trainable params: 1,142,940
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.fit(data,np.array(labels),validation_split=0.1,callbacks=[lr_schedule])




Train on 6822 samples, validate on 759 samples
Epoch 1/1





6822/6822 [==============================] - 248s 36ms/step - loss: 0.6951 - acc: 0.4502 - val_loss: 0.6951 - val_acc: 0.4114


In [0]:
from nltk.tokenize import word_tokenize


In [0]:
# In this method we first separately learn word embeddings and then pass to the embedding layer.
data=[]
data1=df['text'].values.tolist()
#print(data)

In [20]:
nltk.download('punkt')
for line in data1:
  tokens=word_tokenize(line)
  tokens=[w.lower() for w in tokens]
  #table=str.maketrans('','',string.punctuation)
  stripped=[w.translate(tokens) for w in tokens]
  words=[word for word in stripped if word.isalpha()]
  stop_words=set(stopwords.words('english'))
  words=[w for w in words if not w in stop_words]
  data.append(words)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [21]:
print(data)

[['look', 'friend', 'lightyear', 'sohalikut', 'trend', 'play', 'year', 'challeng', 'facebook', 'imgflip', 'com'], ['best', 'yearchalleng', 'complet', 'less', 'year', 'kudus', 'narendramodi', 'jan', 'mumbai', 'india'], ['sam', 'thorn', 'strippin', 'follow', 'follow', 'saw', 'everyon', 'post', 'pic', 'mine', 'jan', 'retweet', 'like'], ['year', 'challeng', 'sweet', 'dee', 'edit'], ['year', 'challeng', 'filter', 'hilari', 'year', 'challeng', 'meme', 'year', 'challeng'], ['get', 'car', 'stranger', 'meet', 'peopl', 'internet', 'alon', 'memecent', 'com', 'uber', 'order', 'stranger', 'internet', 'get', 'car', 'alon'], ['year', 'challeng', 'lit', 'erg', 'wow', 'transform'], ['year', 'challeng', 'emot', 'edit', 'boredpanda', 'com'], ['difficult', 'challeng', 'life', 'year', 'diet', 'challeng', 'haggyaapa'], ['let', 'see', 'ear'], ['fornit', 'die', 'month', 'minecraft', 'never', 'die', 'year', 'proud', 'communiti'], ['facebook', 'year', 'challeng', 'ploy', 'simpl', 'meme', 'exo', 'minh', 'jan'], 

In [0]:
import gensim
model=gensim.models.Word2Vec(sentences=data,size=100,window=5,workers=4,min_count=1)

In [0]:
words=list(model.wv.vocab)
#print(words)

In [24]:
filename=BASE_DIR+'word2vec.txt'
model.wv.save_word2vec_format(filename,binary=False)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
embeddings_index={}
f=open(filename)
for line in f:
  values=line.split()
  word=values[0]
  coefs=np.asarray(values[1:])
  embeddings_index[word]=coefs
f.close()

In [0]:
tokenizer_obj=Tokenizer()
tokenizer_obj.fit_on_texts(data)
sequences=tokenizer_obj.texts_to_sequences(data)
data_pad=pad_sequences(sequences,maxlen=max_length,padding='post')
word_index=tokenizer_obj.word_index
#print(word_index)

In [0]:
embedding_matrix = np.zeros((len(word_index), 100))
for word, index in word_index.items():
    if index > len(word_index) - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector
#print(embedding_matrix)

In [0]:
from keras.initializers import Constant
model=Sequential()
embedding_layer=Embedding(vocab_size,100,embeddings_initializer=Constant(embedding_matrix),input_length=max_length,trainable=False)
model.add(embedding_layer)
model.add(GRU(units=64,dropout=0.2))
model.add(Dense(4,activation='sigmoid'))
lr_schedule = keras.callbacks.LearningRateScheduler(lambda epoch: 1e-8 * 10**(epoch / 10))
opt = SGD(lr=1e-8, momentum=0.9)
model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])

In [29]:
model.fit(data_pad,np.array(labels),validation_split=0.1,callbacks=[lr_schedule])

Train on 6822 samples, validate on 759 samples
Epoch 1/1
6822/6822 [==============================] - 213s 31ms/step - loss: 0.6931 - acc: 0.6725 - val_loss: 0.6931 - val_acc: 0.6887
